In [5]:
#Prueba de actualizacion - Base de dato DNRPA
# Carga de librerias

import datetime
import mysql.connector
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import openpyxl
import pandas as pd
from datetime import datetime
import os

#Variables globales
host = '172.17.22.10'
user = 'Ivan'
password = 'Estadistica123'
database = 'prueba1'

#DF obtenido del sitio DRPNA
df_web = pd.DataFrame() 

In [6]:
# Conexion con BDD 
conn = mysql.connector.connect( host=host, user=user, password=password, database=database)

#Instancia manejadora de BDD

cursor = conn.cursor()

In [7]:
#Busqueda por nombre de TABLA 

table_name = "dnrpa_inscripcion_corrientes_auto"

#Creacion de orden en lenguaje MYSQL
select_dates_query = "SELECT * FROM dnrpa_inscripcion_corrientes_auto"

#Obtencion del DATAFRAME - uso de Orden creada(MySQL) y Conexion(MySQL).
df = pd.read_sql(select_dates_query,con=conn)


#Busqueda del año mas grande (actual o anterior) --> Aclaracion por posible conflicto en los avisos por cada año nuevo
año_de_comparacion = int(df['Fecha'].max().year)

#Conversion a tipo de fecha - necesario para el tratamiento
df['Fecha'] = pd.to_datetime(df['Fecha'])

#Toma de año actual - Se usara para comparar los dataframes
df_año_actual = df[df['Fecha'].dt.year == año_de_comparacion]
print(df_año_actual)



         Fecha  Bella_vista  Corrientes_N1  Curuzu_Cuatia  Goya  Mercedes  \
108 2023-01-01         66.0          106.0           21.0  70.0      30.0   
109 2023-02-01         46.0           42.0           11.0  30.0      17.0   
110 2023-03-01         45.0           69.0           21.0  36.0      19.0   
111 2023-04-01         34.0           69.0            6.0  43.0      17.0   
112 2023-05-01         38.0           51.0           16.0  50.0      11.0   
113 2023-06-01         40.0           55.0           12.0  46.0      18.0   
114 2023-07-01          0.0            0.0            0.0   0.0       0.0   
115 2023-08-01          0.0            0.0            0.0   0.0       0.0   
116 2023-09-01          0.0            0.0            0.0   0.0       0.0   
117 2023-10-01          0.0            0.0            0.0   0.0       0.0   
118 2023-11-01          0.0            0.0            0.0   0.0       0.0   
119 2023-12-01          0.0            0.0            0.0   0.0       0.0   

C:\Users\Usuario\AppData\Local\Temp\ipykernel_608\3318847743.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(select_dates_query,con=conn)


In [10]:
#Datos DNRPA 

try:

    #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ SELENIUM ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
    driver = webdriver.Chrome()
    driver.get('https://www.dnrpa.gov.ar/portal_dnrpa/estadisticas/rrss_tramites/tram_prov.php?origen=portal_dnrpa&tipo_consulta=inscripciones')

    # Obtener la ventana actual
    ventana_actual = driver.current_window_handle
    
    elemento = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/table/tbody/tr[2]/td/select')
    # Obtener todas las opciones del elemento select
    opciones = elemento.find_elements(By.TAG_NAME, 'option')

    # Buscar la opción deseada por su valor y hacer clic en ella
    valor_deseado = '2023'  # Valor de la opción que deseas seleccionar

    for opcion in opciones:
        if opcion.get_attribute('value') == valor_deseado:
            opcion.click()
            break
    
    boton = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/table/tbody/tr[4]/td/input[1]')
    boton.click()
    
    time.sleep(5)
    
    boton_aceptar = driver.find_element(By.XPATH, '//*[@id="seleccion"]/center/center/input')
    boton_aceptar.click()
    
    # Esperar un momento para que se abra la nueva pestaña
    driver.implicitly_wait(5)
    # Cambiar al contexto de la nueva pestaña
    for ventana in driver.window_handles:
        if ventana != ventana_actual:
            driver.switch_to.window(ventana)
    
    time.sleep(5)
    
    # Encontrar el elemento del enlace por el texto visible completo
    enlace = driver.find_element(By.LINK_TEXT, "CORRIENTES")

    # O encontrar el elemento del enlace por el texto visible parcial
    # enlace = driver.find_element_by_partial_link_text("CORRIENTES")

    # Hacer clic en el enlace
    enlace.click()
    # Esperar un momento para que se abra la nueva pestaña
    driver.implicitly_wait(5)
    # Cambiar al contexto de la nueva pestaña
    for ventana in driver.window_handles:
        if ventana != ventana_actual:
            driver.switch_to.window(ventana)
    
    #↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓ ENCONTRAR Y TOMAR LOS DATOS ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
    # Encontrar el elemento <div> con la clase 'grid'
    elemento_div = driver.find_element(By.CLASS_NAME, 'grid')

    # Encontrar la tabla dentro del elemento <div>
    elemento_tabla = elemento_div.find_element(By.TAG_NAME, 'table')

    # Obtener todas las filas de la tabla
    filas = elemento_tabla.find_elements(By.TAG_NAME, 'tr')

    # Lista para almacenar los datos de la tabla
    tabla_datos = []
    
    for fila in filas:
        # Obtener las celdas de cada fila, excluyendo la última columna y la última celda de encabezado
        celdas = fila.find_elements(By.TAG_NAME, 'th') + fila.find_elements(By.TAG_NAME, 'td')[:-1]

        # Lista para almacenar los valores de cada fila
        fila_datos = []

        for celda in celdas:
            valor = celda.text
            if isinstance(valor, str):
                # Verificar si el valor comienza con un número
                if valor.strip() and valor[0].isdigit():
                    try:
                        # Reemplazar el punto decimal por una coma (si es necesario)
                        valor = valor.replace('.', '')
                        # Intentar convertir el valor a float
                        valor = float(valor)
                        print("valor1: ", valor)
                    except ValueError:
                        pass  # Mantener el valor original si no se puede convertir a float
            fila_datos.append(valor)
        print("aca: ", fila_datos)
        # Verificar si la última celda es "Total" y eliminarla
        if fila_datos and fila_datos[-1] == "Total":
            fila_datos.pop()


        tabla_datos.append(fila_datos) 
    
    datos_sin_segunda_fila = tabla_datos[0:1] + tabla_datos[2:]
    # Transponer los datos utilizando pandas
    df = pd.DataFrame(datos_sin_segunda_fila)
    df_transpuesta = df.transpose()
    df_transpuesta = df_transpuesta.drop(df_transpuesta.index[-1])
    df_transpuesta = df_transpuesta.drop(df_transpuesta.columns[-1],axis=1)
    
    
    #Conversion de MESES a formato Y-M-D , tipo de dato: datetime
    print(df_transpuesta[0][1:])
    meses = df_transpuesta[0][1:]

    #Donde almacenamos las nuevas fechas
    nuevas_fechas = list()

    for i in range(1, len(meses)+1):

        if i < 10:
            fecha_str =  '01-0'+str(i)+"-"+ str(valor_deseado)
        else:
            fecha_str = '01-'+str(i)+"-"+ str(valor_deseado)

        fecha_str = datetime.strptime(fecha_str,'%d-%m-%Y').date()
        nuevas_fechas.append(fecha_str)

    #Reasignacion de fechas
    df_transpuesta[0][1:] = nuevas_fechas
    
    df_web = df_transpuesta

except Exception as e:
    
    # Manejar cualquier excepción ocurrida durante la carga de datos
    print(f"Registro automotor: Ocurrió un error durante la carga de datos: {str(e)}")
    conn.close()  # Cerrar la conexión en caso de error

aca:  ['RRSS / Mes', 'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic', 'Total']
aca:  []
valor1:  66.0
valor1:  46.0
valor1:  45.0
valor1:  34.0
valor1:  38.0
valor1:  40.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
aca:  ['5001 - BELLA VISTA', 66.0, 46.0, 45.0, 34.0, 38.0, 40.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
valor1:  106.0
valor1:  42.0
valor1:  69.0
valor1:  69.0
valor1:  51.0
valor1:  55.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
aca:  ['5002 - CORRIENTES N° 1', 106.0, 42.0, 69.0, 69.0, 51.0, 55.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
valor1:  21.0
valor1:  11.0
valor1:  21.0
valor1:  6.0
valor1:  16.0
valor1:  12.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
valor1:  0.0
aca:  ['5003 - CURUZU CUATIA', 21.0, 11.0, 21.0, 6.0, 16.0, 12.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
valor1:  70.0
valor1:  30.0
valor1:  36.0
valor1:  43.0
valor1:  50.0
valor1:  46.0
valor1:  0.0
val

In [16]:
print(df_web)

print("""
        ********************************************************************************************************
      """)

print(df_año_actual)

            0                   1                       2   \
0   RRSS / Mes  5001 - BELLA VISTA  5002 - CORRIENTES N° 1   
1   2023-01-01                66.0                   106.0   
2   2023-02-01                46.0                    42.0   
3   2023-03-01                45.0                    69.0   
4   2023-04-01                34.0                    69.0   
5   2023-05-01                38.0                    51.0   
6   2023-06-01                40.0                    55.0   
7   2023-07-01                 0.0                     0.0   
8   2023-08-01                 0.0                     0.0   
9   2023-09-01                 0.0                     0.0   
10  2023-10-01                 0.0                     0.0   
11  2023-11-01                 0.0                     0.0   
12  2023-12-01                 0.0                     0.0   

                      3            4                5   \
0   5003 - CURUZU CUATIA  5004 - GOYA  5005 - MERCEDES   
1              